In [12]:
import pandas as pd
from pathlib import Path
from scipy.stats import ttest_ind, mannwhitneyu
import matplotlib.pyplot as plt
import numpy as np

In [13]:
FILE_PREFIXES = {
    "cbench": "cbench_stat_results_on_",
    "mibench": "mibench_stat_results_on_",
}

P_VALUE_THRESHOLD = 0.05

In [8]:
def prepare_data_for_plotting(suite_name, prefix):
    """
    Собирает все необходимые данные для построения графика из файлов.
    Возвращает DataFrame, готовый для визуализации.
    """
    try:
        df_on_x86 = pd.read_csv(f"{prefix}x86_64.csv")
        df_on_arm = pd.read_csv(f"{prefix}arm64.csv")
    except FileNotFoundError as e:
        print(f"ПРЕДУПРЕЖДЕНИЕ: Не найден файл для {suite_name}: {e}. Пропускаем.")
        return None

    common_benchmarks = sorted(list(
        set(df_on_x86['benchmark'].unique()) & set(df_on_arm['benchmark'].unique())
    ))

    plot_data = []

    for benchmark_name in common_benchmarks:
        # --- Анализ на платформе x86_64 ---
        native_x86 = df_on_x86[(df_on_x86['benchmark'] == benchmark_name) & (df_on_x86['flags_source_arch'] == 'x86_64')]['exec_time']
        foreign_x86 = df_on_x86[(df_on_x86['benchmark'] == benchmark_name) & (df_on_x86['flags_source_arch'] == 'arm64')]['exec_time']
        
        if not (native_x86.empty or foreign_x86.empty):
            _, p_value = mannwhitneyu(native_x86, foreign_x86, alternative='less', nan_policy='omit')
            # Потеря = (чужое время - свое время) / свое время
            loss = (foreign_x86.median() - native_x86.median()) / native_x86.median() * 100
            plot_data.append({
                "benchmark": benchmark_name,
                "test_platform": "x86_64",
                "p_value": p_value,
                "performance_loss_percent": loss
            })

        # --- Анализ на платформе arm64 ---
        native_arm = df_on_arm[(df_on_arm['benchmark'] == benchmark_name) & (df_on_arm['flags_source_arch'] == 'arm64')]['exec_time']
        foreign_arm = df_on_arm[(df_on_arm['benchmark'] == benchmark_name) & (df_on_arm['flags_source_arch'] == 'x86_64')]['exec_time']

        if not (native_arm.empty or foreign_arm.empty):
            _, p_value = mannwhitneyu(native_arm, foreign_arm, alternative='less', nan_policy='omit')
            loss = (foreign_arm.median() - native_arm.median()) / native_arm.median() * 100
            plot_data.append({
                "benchmark": benchmark_name,
                "test_platform": "arm64",
                "p_value": p_value,
                "performance_loss_percent": loss
            })
            
    return pd.DataFrame(plot_data)

In [16]:
def plot_significance_matrix(df, suite_name):
    """
    Строит матрицу значимости, где цвет ячейки зависит от p-value,
    а текст показывает p-value и практический эффект.
    """
    if df is None or df.empty:
        return

    # Создаем сводную таблицу для удобства
    p_value_pivot = df.pivot(index="benchmark", columns="test_platform", values="p_value")
    loss_pivot = df.pivot(index="benchmark", columns="test_platform", values="performance_loss_percent")

    benchmarks = p_value_pivot.index
    platforms = p_value_pivot.columns
    
    # Создаем матрицу цветов: 1 для значимого, 0 для незначимого
    significance_matrix = (p_value_pivot < P_VALUE_THRESHOLD).astype(int)

    fig, ax = plt.subplots(figsize=(8, len(benchmarks) * 0.6 + 1))
    
    # Используем красно-зеленую палитру: 0 (незначимо) -> зеленый, 1 (значимо) -> красный
    cmap = plt.cm.colors.ListedColormap(['#90EE90', '#F08080']) # LightGreen, LightCoral
    ax.imshow(significance_matrix, cmap=cmap, aspect='auto')

    # Добавляем текст в ячейки
    for i, bench in enumerate(benchmarks):
        for j, platform in enumerate(platforms):
            p_val = p_value_pivot.loc[bench, platform]
            loss = loss_pivot.loc[bench, platform]
            
            # Формируем текст
            # Если потеря отрицательная, значит "чужие" флаги оказались лучше!
            sign = "+" if loss >= 0 else ""
            text = f"p={p_val:.3f}\nLoss: {sign}{loss:.2f}%"
            
            ax.text(j, i, text, ha='center', va='center', color='black', fontsize=10)

    # Настраиваем оси
    ax.set_xticks(np.arange(len(platforms)))
    ax.set_yticks(np.arange(len(benchmarks)))
    ax.set_xticklabels([f"Тест на {p}" for p in platforms])
    ax.set_yticklabels(benchmarks)
    
    plt.setp(ax.get_xticklabels(), rotation=0, ha="center", rotation_mode="anchor")
    
    # Добавляем легенду
    legend_elements = [
        plt.Rectangle((0, 0), 1, 1, color='#F08080', label=f'p < {P_VALUE_THRESHOLD} (Непереносимо)'),
        plt.Rectangle((0, 0), 1, 1, color='#90EE90', label=f'p >= {P_VALUE_THRESHOLD} (Переносимо)')
    ]
    ax.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left')

    ax.set_title(f"Матрица значимости непереносимости флагов ({suite_name})")
    fig.tight_layout()
    
    # Сохраняем график
    output_path = Path(f"plots/significance_matrix_{suite_name}.png")
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    print(f"График сохранен в файл: {output_path}")
    plt.close()


In [17]:
plot_df = prepare_data_for_plotting("mibench", "mibench_stat_results_on_")
plot_significance_matrix(plot_df, "mibench")

График сохранен в файл: plots/significance_matrix_mibench.png
